<a href="https://colab.research.google.com/github/Akshata1992/Implement_ResearchPaper/blob/main/AlexNet_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing AlexNet 

In [ ]:
#Import all necessary libraries
import keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Conv2D,Flatten,Dropout,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.losses import CategoricalCrossentropy
import numpy as np

In [ ]:
#Initialize random seed
np.random.seed(1000)

The network consists of a kernel or filters with size 11 x 11, 5 x 5, 3 x 3, 3 x 3 and 3 x 3 for its five convolutional layers respectively,3 fully connected layers, ReLU as an activation function at all layers except at the output layer

In [ ]:
#Instantiate the model
AlexNet = Sequential()

#Conv Layer -1 
AlexNet.add(Conv2D(filters=96,kernel_size=(3,3),strides=(4,4),input_shape=(32,32,3), activation='relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))

#Conv Layer - 2
AlexNet.add(Conv2D(256,kernel_size=(5,5),strides=(1,1),padding='same',activation='relu'))
AlexNet.add(BatchNormalization())
AlexNet.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))

#Conv Layer - 3
AlexNet.add(Conv2D(384,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))

#Conv Layer - 4
AlexNet.add(Conv2D(384,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))

#Conv Layer - 5
AlexNet.add(Conv2D(256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
AlexNet.add(BatchNormalization())
AlexNet.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))

#Flatten the layers
AlexNet.add(Flatten())

#Fully Connected layer - 1
AlexNet.add(Dense(4096))
AlexNet.add(Activation('relu'))
#Add drop out to prevent overfitting
AlexNet.add(Dropout(0.4))

#Fully Connected layer - 2
AlexNet.add(Dense(4096))
AlexNet.add(Activation('relu'))
#Add drop out to prevent overfitting
AlexNet.add(Dropout(0.4))

#Output layer
AlexNet.add(Dense(10))
AlexNet.add(Activation('softmax'))

AlexNet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 8, 8, 96)          2688      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 256)         1024      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 256)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 2, 2, 384)         885120    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 2, 384)        

In [ ]:
#Compile the model
AlexNet.compile(optimizer='SGD',loss='CategoricalCrossentropy',metrics=['accuracy'])

In [ ]:
#Keras library for CIFAR10 dataset
from keras.datasets import cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [ ]:
# train test split using sklearn 
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.3)

In [ ]:
#dimention of cifar dataset
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(35000, 32, 32, 3)
(15000, 32, 32, 3)
(35000, 1)
(15000, 1)


In [ ]:
#One hot encoding the labels
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(35000, 10)
(15000, 10)
(10000, 10)


In [ ]:
#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rotation_range=2,horizontal_flip=True,zoom_range=0.1)
val_generator = ImageDataGenerator(rotation_range=2,horizontal_flip=True,zoom_range=0.1)
test_generator = ImageDataGenerator(rotation_range=2,horizontal_flip=True,zoom_range=0.1)

#Fit the dataset to transformation
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [ ]:
#Learning rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr = ReduceLROnPlateau(monitor='val_acc',factor=0.01,patience=3,min_lr=1e-5)

In [ ]:
#Defining parameters for model fitting
batch_size = 100
epochs = 200
learning_rate = 0.001

In [ ]:
#Fit the model
AlexNet.fit_generator(train_generator.flow(x_train,y_train,batch_size=batch_size),epochs=epochs,steps_per_epoch=x_train.shape[0]//batch_size,
                      validation_data=val_generator.flow(x_val,y_val,batch_size=batch_size),validation_steps=250,callbacks=[lrr],verbose=1)

Epoch 1/200
350/350 [==============================] - 23s 65ms/step - loss: 1.8495 - accuracy: 0.3189 - val_loss: 1.8938 - val_accuracy: 0.3645
Epoch 2/200
350/350 [==============================] - 17s 48ms/step - loss: 1.5323 - accuracy: 0.4400
Epoch 3/200
350/350 [==============================] - 17s 50ms/step - loss: 1.4019 - accuracy: 0.4952
Epoch 4/200
350/350 [==============================] - 17s 48ms/step - loss: 1.3251 - accuracy: 0.5242
Epoch 5/200
350/350 [==============================] - 17s 47ms/step - loss: 1.2646 - accuracy: 0.5478
Epoch 6/200
350/350 [==============================] - 17s 47ms/step - loss: 1.2072 - accuracy: 0.5656
Epoch 7/200
350/350 [==============================] - 17s 47ms/step - loss: 1.1658 - accuracy: 0.5854
Epoch 8/200
350/350 [==============================] - 17s 47ms/step - loss: 1.1276 - accuracy: 0.6010
Epoch 9/200
350/350 [==============================] - 17s 48ms/step - loss: 1.0973 - accuracy: 0.6082
Epoch 10/200
350/350 [=========

In [ ]:
#Visualize the performance
import matplotlib.pyplot as plt

#plotting loss and accuracy
f,ax = plt.subplots(2,1)

#loss graph
ax[0].plot(AlexNet.history.history['loss'],color='b',label='Training Loss')
ax[0].plot(AlexNet.history.history['val_loss'],color='r',label='Validation Loss')

#Accuracy graph
ax[1].plot(AlexNet.history.history['accuracy'],color='b',label='Training Accuracy')
ax[1].plot(AlexNet.history.history['val_accuracy'],color='r',label='Validation Accuracy')

plt.legend()

In [ ]:
# Plotting the Confusion Matrics
def plot_confusion_matrix(y_true,y_pred,
                           classes,normalize=False,
                           title=None,
                           cmap=plt.cm.Blues):
  
  if not title:
    if normalize:
      title = 'Normalized Confusion Matrics'
    else:
      title = 'Non-Normalized Confusion Matrics'

  cm = confusion_matrix(y_true,y_pred)
  if normalize:
    cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    print('Normalized Confusion Matrix')
  else:
    print('Non-Normalized Confusion matrix')

  #print the confusion matrix
  fig,ax = plt.subplots(figsize=(7,7))
  im = ax.imshow(cm,interpolation='nearest',cmap=cmap)
  ax.figure.colorbar(im,ax=ax)

  ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

  # Rotate the tick labels and set their alignment.
  plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
          ax.text(j, i, format(cm[i, j], fmt),
                  ha="center", va="center",
                  color="white" if cm[i, j] > thresh else "black")
  fig.tight_layout()
  return ax

np.set_printoptions(precision=2)

In [ ]:
#Making predictions for test images
y_pred = AlexNet.predict_classes(x_test)
y_true = np.argmax(y_test,axis=1)

#Plot the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_mtx=confusion_matrix(y_true,y_pred)

class_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Plotting non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes = class_names,title = 'Confusion matrix, without normalization')

# Plotting normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes=class_names, normalize=True, title='Normalized confusion matrix')

In [ ]:
#Classification Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy Score : ",accuracy_score(y_true,y_pred))